In [1]:
#r "nuget:Google.OrTools"

using Google.OrTools.LinearSolver;
using System.Linq; 

Installing package Google.OrTools................done!

Successfully added reference to package Google.OrTools, version 7.4.7247

In [2]:
//We prepare a function to call the solver solve. Code reuse!
public static void SolveAndPrint(this Solver solver)
{
    var resultStatus = solver.Solve();

    // Check that the problem has an optimal solution.
    if (resultStatus != Solver.ResultStatus.OPTIMAL)
    {
        Console.WriteLine("The problem does not have an optimal solution!");
        return;
    }

    Console.WriteLine("Problem solved in " + solver.WallTime() + " milliseconds");

    // The objective value of the solution.
    Console.WriteLine("Optimal objective value = " + solver.Objective().Value());

    display(solver.variables().Select(a => new { Name  = a.Name(), Value = a.SolutionValue() }));
}

# Applications of linear programming

## Wire production

A plant makes aluminium and copper wires. Each Kg of aluminium wire requires 10 kWh of electricity and $\frac{1}{2}$ hour of labour. Each Kg of Copper wire requires 4 kWh of electricity and $1$ hour of labour. Total weight delivered to the plant daily is limited to 56 Kg. Electricity is limited to 450 kWh/day, labour is limited to 42.5 hours/day at a cost of 11 € an hour, Electricity cost is 20 € / MWh, Aluminium cost is 1.8 €/Kg, Copper cost is 5.4 €/Kg.  Aluminium wire sales price is 45 €/Kg, Copper wire sales price is 50 €/Kg. 

What should be produced to maximise profit and what is the maximum profit? 

We want to setup the objective function first and we will need variables for the weight of aluminium $Al$ and copper $Cu$ used in the plant, $El$ is electricity, $La$ is labour. All the ${}_{Purchase}$ and ${}_{Sale}$ are the direct values given in the problem definition.

$$\text{Obj:} \max ( Al * (Al_{Sale} - Al_{Purchase}) + Cu * (Cu_{Sale} - Cu_{Purchase}) - El_{Purchase} (\frac{1}{100} * Al + \frac{4}{1000} * Cu) - La_{Purchase}(\frac{1}{2} * Al + 1 * Cu) )$$

Now we want to setup the various constraints that the problem is subject to:

Electricity usage limit:

$$10 * Al + 4 * Cu \leq 450$$

Labour usage limit:

$$\frac{1}{2} * Al + 1 * Cu \leq 42.5$$

Weight limits: 

$$Al + Cu \leq 56$$

In the following block, the problem is definde for the solver:

In [3]:
Solver solver = Solver.CreateSolver("LinearProgramming", "CLP_LINEAR_PROGRAMMING");

//Setting up the variables and constants of the problem 
Variable Al = solver.MakeNumVar(0.0, double.PositiveInfinity, "Al");
Variable Cu = solver.MakeNumVar(0.0, double.PositiveInfinity, "Cu");
double El_Price = 20.0; double El_Max = 450; 
double La_Price = 11.0; double La_Max = 42.5; 
double Weight_Max = 56.0; 
double Al_Purchase = 1.8 ; double Al_Sale = 45.0;
double Cu_Purchase = 5.4; double Cu_Sale = 50.0;

// Maximize revenue 
Objective objective = solver.Objective();
objective.SetCoefficient(Al, Al_Sale-Al_Purchase-(La_Price * 1/2.0)-(El_Price * 10/1000.0));
objective.SetCoefficient(Cu, Cu_Sale-Cu_Purchase-(La_Price * 1)-(El_Price * 4/1000.0));
objective.SetMaximization();

// Electricity usage limit
Constraint c0 = solver.MakeConstraint(0, El_Max);
c0.SetCoefficient(Al, 10);
c0.SetCoefficient(Cu, 4);

// Labour usage limit
Constraint c1 = solver.MakeConstraint(0, La_Max);
c1.SetCoefficient(Al, 1/2.0);
c1.SetCoefficient(Cu, 1);

// Weight limit
Constraint c2 = solver.MakeConstraint(0, Weight_Max);
c2.SetCoefficient(Al, 1);
c2.SetCoefficient(Cu, 1);

SolveAndPrint(solver);

Problem solved in 4 milliseconds
Optimal objective value = 2027,0333333333333


index,Name,Value
0,Al,37.666666666666664
1,Cu,18.333333333333336



### Q1: Which limit is the most restrictive? 

Try changing the limits of the constraints to find which constraints are the most restrictive. You could also calculate the added value to the objective function of 1 unit of the contraint. It is also possible to get this information out of the dual formulation of the solved linear program. 

You can lookup up the solution's dual value and reduced cost for each variable. 

In [4]:
//Find which constraints are the most restrictive


### Q2: How does the problem change if we have a 2% mass loss when making Copper wire? 

A1: We can either add a new variable for the finished Copper wire and an equation $Cu_{out}=0.98Cu_{in}$ and then that variable in the objective function. 

$$\text{Obj:}  \max ( Al * (Al_{Sale} - Al_{Purchase}) + (Cu_{out} * Cu_{Sale}) - (Cu_{in} *Cu_{Purchase}) - El_{Purchase} (\frac{1}{100} * Al + \frac{4}{1000} * Cu) - La_{Purchase}(\frac{1}{2} * Al + 1 * Cu) )$$

$$st: \qquad Cu_{out}=0.98 * Cu_{in}$$

A2: Because nothing apart from the objective function depends on the output, we can simply adapt the objective function by multiplying the copper variable in the objective function by 0.98 prior to multiplying it by the value of the finished copper wire. 

$$\text{Obj:}  \max ( Al * (Al_{Sale} - Al_{Purchase}) + Cu * (0.98 * Cu_{Sale} - Cu_{Purchase}) - El_{Purchase} (\frac{1}{100} * Al + \frac{4}{1000} * Cu) - La_{Purchase}(\frac{1}{2} * Al + 1 * Cu) )$$

In [5]:
//Modify the objective function

### Q3: How does the problem change if the labor is now a take or pay contract? I.e. we Must now pay for the 42.5 hours irrespective of if we use them fully or not? 
A: We still keep the equation limiting labour, but now in the objective function, the price per hour of labor and the hours worked are both constants, so they can be directly multiplied and left as a constant.  

$$Obj: \max ( Al * (Al_{Sale} - Al_{Purchase}) + Cu * (Cu_{Sale} - Cu_{Purchase}) - El_{Purchase} (\frac{1}{100} * Al + \frac{4}{1000} * Cu) - La_{Purchase} * 42.5 ) $$ 

Alternatively, we can simply remove it from the objective function since it is constant anyways, but then you have to remember to substract it afterwards to get the profit. 

In [6]:
//Modify the objective problem

### Answers 
<details>
<summary>A1: Which limit is the most restrictive? </summary>
The weight limit is the largest constraint limiting the optimal value (+30.86 in objective value for one unit of weight more), followed by the electricity limit (+0.66 for one unit of electricity). 
</details>